# Preprocessing Pipeline

## Import Statements

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np

## Functions

In [2]:
def remove_outliers(df, group_by="TaskName", value_col="price"):
    def filter_group(group):
        Q1 = group[value_col].quantile(0.25)
        Q3 = group[value_col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        return group[(group[value_col] >= lower) & (group[value_col] <= upper)]
    
    return df.groupby(group_by, group_keys=False).apply(filter_group)

## Read in Data

In [3]:
# CPI for price adjustment
cpi_index = pd.read_csv('data/cpi.csv')
cpi_index.head()

,Quarter,CPI
0,2021Q1,117.9
1,2021Q2,118.8
2,2021Q3,119.7
3,2021Q4,121.3
4,2022Q1,123.9


In [4]:
# Booking/Ticket data
df = pd.read_csv('data/data.csv', encoding = 'windows-1252')
df.head()

,ï»¿FCID,BookingID,BCreatedDateAEST,BStatusAfterSubmitted,BStatusFromDateTimeAEST,BStatusFinal,BTicketID,BTicketType,TaskName,PriceIncGSTRaw,BOdoNum,BOdoText,cVMake,cVMakeModel,cVYear,idFuel,idLitres,idTransmission,idDrive,idIsHybrid,VMid,BShopID,BShopPostcode,BShopState,BShopRegionName,BShopRegionClass,IsCustomService,IsCustomRepair,IsDeleted
0,4,2172785,19/02/2025,33. Approved,4/04/2025 8:52,64. Completed,3647415,Repair,Tail Light Assembly Replacement,665.00,105291.0,"105,291 km",TOYOTA,TOYOTA LANDCRUISER PRADO,2020,Diesel,2.8,Auto,4WD,0.0,TOY38983,16764,2500,NSW,Wollongong,2,0,0,0
1,4,2172921,19/02/2025,33. Approved,20/02/2025 10:41,64. Completed,3647828,Prescribed,"Prescribed - 60,000 kilometres / 24 months",939.50,19957.0,"19,957 km",ISUZU,ISUZU N-SERIES,2020,Diesel,NaN,Auto,2WD,NaN,NaN,16955,2790,NSW,South Eastern,2,0,0,0
2,9,2173119,19/02/2025,33. Approved,19/02/2025 13:42,64. Completed,3648072,Log,"Logbook Service - 135,000 km / 108 months",371.82,141320.0,"141,320 km",HYUNDAI,HYUNDAI TUCSON,2021,Petrol,2.0,Auto,NaN,0.0,HYU48327,20684,5554,SA,Yorke Peninsula,2,0,0,0
3,4,2173187,19/02/2025,33. Approved,19/02/2025 13:22,64. Completed,3648235,Repair,Roadworthy Inspection / Pink Slips,50.78,227817.0,"227,817 km",ISUZU,ISUZU D-MAX,2015,Diesel,3.0,Auto,2WD,0.0,ISU30317,2108,2208,NSW,Sydney,1,0,0,0
4,2,2173202,19/02/2025,33. Approved,19/02/2025 13:31,64. Completed,3648252,Repair,Balance Tyres,31.90,14528.0,"14,528 km",SUBARU,SUBARU FORESTER,2024,Petrol,2.5,Auto,4WD,0.0,SUB47068,17961,3844,VIC,Traralgon,2,0,0,0


## Rename Columns

In [5]:
df = df.rename(columns={
    'cVMake': 'Make',
    'cVMakeModel': 'Model',
    'cVYear': 'Year',
    'idFuel': 'FuelType',
    'idLitres': 'EngineSize',
    'idTransmission': 'Transmission',
    'idDrive': 'DriveType',
    'idIsHybrid': 'IsHybrid',
    'BOdoNum' : 'Odometer',
    })

## Drop Columns that will not be used

In [6]:
columns_to_drop = ['ï»¿FCID', 'BOdoText', 'VMid']
df.drop(columns = columns_to_drop, inplace=True)
df.head()

,BookingID,BCreatedDateAEST,BStatusAfterSubmitted,BStatusFromDateTimeAEST,BStatusFinal,BTicketID,BTicketType,TaskName,PriceIncGSTRaw,Odometer,Make,Model,Year,FuelType,EngineSize,Transmission,DriveType,IsHybrid,BShopID,BShopPostcode,BShopState,BShopRegionName,BShopRegionClass,IsCustomService,IsCustomRepair,IsDeleted
0,2172785,19/02/2025,33. Approved,4/04/2025 8:52,64. Completed,3647415,Repair,Tail Light Assembly Replacement,665.00,105291.0,TOYOTA,TOYOTA LANDCRUISER PRADO,2020,Diesel,2.8,Auto,4WD,0.0,16764,2500,NSW,Wollongong,2,0,0,0
1,2172921,19/02/2025,33. Approved,20/02/2025 10:41,64. Completed,3647828,Prescribed,"Prescribed - 60,000 kilometres / 24 months",939.50,19957.0,ISUZU,ISUZU N-SERIES,2020,Diesel,NaN,Auto,2WD,NaN,16955,2790,NSW,South Eastern,2,0,0,0
2,2173119,19/02/2025,33. Approved,19/02/2025 13:42,64. Completed,3648072,Log,"Logbook Service - 135,000 km / 108 months",371.82,141320.0,HYUNDAI,HYUNDAI TUCSON,2021,Petrol,2.0,Auto,NaN,0.0,20684,5554,SA,Yorke Peninsula,2,0,0,0
3,2173187,19/02/2025,33. Approved,19/02/2025 13:22,64. Completed,3648235,Repair,Roadworthy Inspection / Pink Slips,50.78,227817.0,ISUZU,ISUZU D-MAX,2015,Diesel,3.0,Auto,2WD,0.0,2108,2208,NSW,Sydney,1,0,0,0
4,2173202,19/02/2025,33. Approved,19/02/2025 13:31,64. Completed,3648252,Repair,Balance Tyres,31.90,14528.0,SUBARU,SUBARU FORESTER,2024,Petrol,2.5,Auto,4WD,0.0,17961,3844,VIC,Traralgon,2,0,0,0


## Remove Rows


In [7]:
# Remove custom services and repairs
df = df[df['IsCustomService'] != '1']
df = df[df['IsCustomRepair'] != '1']
df = df[df['TaskName'] != 'Custom Repair']

# Remove products and tyres as they are unpredictable without quantity information 
df = df[df['TaskName'] != '((Products))']
df = df[df['TaskName'] != '((Tyres))']
df = df[df['TaskName'] != 'Tyre Replacement']

# Remove Rare ticket types
df = df[df['BTicketType'] != 'OtherTicket']
df = df[df['BTicketType'] != 'Custom']
df = df[df['BTicketType'] != 'Basic']

# Remove Tickets with zero or negative prices 
df = df[df['PriceIncGSTRaw'] > 0]

## Create Months and Kilometer Columns from TicketType

In [8]:
# Create Columns
df['Distance'] = None
df['Months'] = None

mask = df['BTicketType'] == 'Log'
split_1 = df.loc[mask, 'TaskName'].str.split(' - ', n=1, expand=True)
df.loc[mask, 'TaskName'] = split_1[0]
df.loc[mask, 'Rest'] = split_1[1].fillna('')

split_2 = df.loc[mask, 'Rest'].str.split(' / ', n=1, expand=True)
df.loc[mask, 'Distance'] = split_2[0]
df.loc[mask, 'Months'] = split_2[1]

mask = df['BTicketType'] == 'Capped'
split_1 = df.loc[mask, 'TaskName'].str.split(' - ', n=1, expand=True)
df.loc[mask, 'TaskName'] = split_1[0]
df.loc[mask, 'Distance'] = split_1[1].fillna('')

mask = df['BTicketType'] == 'Prescribed'
split_1 = df.loc[mask, 'TaskName'].str.split(' - ', n=1, expand=True)
df.loc[mask, 'TaskName'] = split_1[0]
df.loc[mask, 'Distance'] = split_1[1].fillna('')

df = df.drop(columns=['Rest'])

In [9]:
# Format as Numeric
df['Months'] = df['Months'].astype(str).str.extract(r'(\d+)')
df['Months'] = pd.to_numeric(df['Months'], errors='coerce')

df['Distance'] = df['Distance'].astype(str).str.replace(',', '', regex=False)  
df['Distance'] = df['Distance'].astype(str).str.extract(r'(\d+)')
df['Distance'] = pd.to_numeric(df['Distance'], errors='coerce')

df.head()

,BookingID,BCreatedDateAEST,BStatusAfterSubmitted,BStatusFromDateTimeAEST,BStatusFinal,BTicketID,BTicketType,TaskName,PriceIncGSTRaw,Odometer,Make,Model,Year,FuelType,EngineSize,Transmission,DriveType,IsHybrid,BShopID,BShopPostcode,BShopState,BShopRegionName,BShopRegionClass,IsCustomService,IsCustomRepair,IsDeleted,Distance,Months
0,2172785,19/02/2025,33. Approved,4/04/2025 8:52,64. Completed,3647415,Repair,Tail Light Assembly Replacement,665.00,105291.0,TOYOTA,TOYOTA LANDCRUISER PRADO,2020,Diesel,2.8,Auto,4WD,0.0,16764,2500,NSW,Wollongong,2,0,0,0,NaN,NaN
1,2172921,19/02/2025,33. Approved,20/02/2025 10:41,64. Completed,3647828,Prescribed,Prescribed,939.50,19957.0,ISUZU,ISUZU N-SERIES,2020,Diesel,NaN,Auto,2WD,NaN,16955,2790,NSW,South Eastern,2,0,0,0,60000.0,NaN
2,2173119,19/02/2025,33. Approved,19/02/2025 13:42,64. Completed,3648072,Log,Logbook Service,371.82,141320.0,HYUNDAI,HYUNDAI TUCSON,2021,Petrol,2.0,Auto,NaN,0.0,20684,5554,SA,Yorke Peninsula,2,0,0,0,135000.0,108.0
3,2173187,19/02/2025,33. Approved,19/02/2025 13:22,64. Completed,3648235,Repair,Roadworthy Inspection / Pink Slips,50.78,227817.0,ISUZU,ISUZU D-MAX,2015,Diesel,3.0,Auto,2WD,0.0,2108,2208,NSW,Sydney,1,0,0,0,NaN,NaN
4,2173202,19/02/2025,33. Approved,19/02/2025 13:31,64. Completed,3648252,Repair,Balance Tyres,31.90,14528.0,SUBARU,SUBARU FORESTER,2024,Petrol,2.5,Auto,4WD,0.0,17961,3844,VIC,Traralgon,2,0,0,0,NaN,NaN


## Grouping / Dropping Rare Tasks 
- Decision must be made here to either drop rare tasks completely or group into a rare category.
- Lots of variance so grouping is probably not a good idea 

In [10]:
task_counts = df['TaskName'].value_counts()

# Define a threshold 
threshold = 100
rare_tasks = task_counts[task_counts < threshold].index


print(len(df['TaskName'].unique()))

# Group rare tasks 
#df['TaskName'] = df['TaskName'].replace(rare_tasks, 'Other')

# Drop rare tasks
df = df[~df['TaskName'].isin(rare_tasks)].copy()

print(len(df['TaskName'].unique()))

1630
480


## Grouping / Dropping Rare Models

In [11]:
task_counts = df['Model'].value_counts()

# Define a threshold 
threshold = 20
rare_tasks = task_counts[task_counts < threshold].index


print(len(df['Model'].unique()))

# Group rare models 
#df['TaskName'] = df['TaskName'].replace(rare_tasks, 'Other')

# Drop rare models
df = df[~df['Model'].isin(rare_tasks)].copy()

print(len(df['Model'].unique()))

1574
574


## Create Adjusted Price based on CPI

In [12]:
# 1. Create date column
df['BCreatedDateAEST'] = pd.to_datetime(df['BCreatedDateAEST'], format='mixed', dayfirst=True);
df['Date'] = df['BCreatedDateAEST'].dt.date

In [13]:
# 2. Price Adjustment
df = df[df['PriceIncGSTRaw'] != 0]
df['Date'] = pd.to_datetime(df['Date']) # ensure 'date' is in datetime format
df['Quarter'] = df['Date'].dt.to_period('Q').astype(str)
df = df.merge(cpi_index, on='Quarter', how='left')
base_cpi = cpi_index['CPI'].iloc[-1]
df['AdjustedPrice'] = round(df['PriceIncGSTRaw'] * (base_cpi / df['CPI']), 2) # round to 2 decimal places for consistency
df['AdjustedPrice'] = round(df['AdjustedPrice'].fillna(df['PriceIncGSTRaw'])) # use original price where CPI is missing (for current quarter which does not need adjustment)

In [14]:
df.head()

,BookingID,BCreatedDateAEST,BStatusAfterSubmitted,BStatusFromDateTimeAEST,BStatusFinal,BTicketID,BTicketType,TaskName,PriceIncGSTRaw,Odometer,Make,Model,Year,FuelType,EngineSize,Transmission,DriveType,IsHybrid,BShopID,BShopPostcode,BShopState,BShopRegionName,BShopRegionClass,IsCustomService,IsCustomRepair,IsDeleted,Distance,Months,Date,Quarter,CPI,AdjustedPrice
0,2172785,2025-02-19,33. Approved,4/04/2025 8:52,64. Completed,3647415,Repair,Tail Light Assembly Replacement,665.00,105291.0,TOYOTA,TOYOTA LANDCRUISER PRADO,2020,Diesel,2.8,Auto,4WD,0.0,16764,2500,NSW,Wollongong,2,0,0,0,NaN,NaN,2025-02-19,2025Q1,140.7,665.0
1,2172921,2025-02-19,33. Approved,20/02/2025 10:41,64. Completed,3647828,Prescribed,Prescribed,939.50,19957.0,ISUZU,ISUZU N-SERIES,2020,Diesel,NaN,Auto,2WD,NaN,16955,2790,NSW,South Eastern,2,0,0,0,60000.0,NaN,2025-02-19,2025Q1,140.7,940.0
2,2173119,2025-02-19,33. Approved,19/02/2025 13:42,64. Completed,3648072,Log,Logbook Service,371.82,141320.0,HYUNDAI,HYUNDAI TUCSON,2021,Petrol,2.0,Auto,NaN,0.0,20684,5554,SA,Yorke Peninsula,2,0,0,0,135000.0,108.0,2025-02-19,2025Q1,140.7,372.0
3,2173187,2025-02-19,33. Approved,19/02/2025 13:22,64. Completed,3648235,Repair,Roadworthy Inspection / Pink Slips,50.78,227817.0,ISUZU,ISUZU D-MAX,2015,Diesel,3.0,Auto,2WD,0.0,2108,2208,NSW,Sydney,1,0,0,0,NaN,NaN,2025-02-19,2025Q1,140.7,51.0
4,2173202,2025-02-19,33. Approved,19/02/2025 13:31,64. Completed,3648252,Repair,Balance Tyres,31.90,14528.0,SUBARU,SUBARU FORESTER,2024,Petrol,2.5,Auto,4WD,0.0,17961,3844,VIC,Traralgon,2,0,0,0,NaN,NaN,2025-02-19,2025Q1,140.7,32.0


## Remove Columns that will not be used

In [15]:
columns_to_drop = [ 'BCreatedDateAEST', 'CPI', 'Quarter', 'PriceIncGSTRaw' , 'BShopID', 'BShopRegionName', 'BShopPostcode', 'IsDeleted', 'BStatusFromDateTimeAEST', 'Date', 'BStatusFinal', ]
df.drop(columns = columns_to_drop, inplace=True)
df.head()

,BookingID,BStatusAfterSubmitted,BTicketID,BTicketType,TaskName,Odometer,Make,Model,Year,FuelType,EngineSize,Transmission,DriveType,IsHybrid,BShopState,BShopRegionClass,IsCustomService,IsCustomRepair,Distance,Months,AdjustedPrice
0,2172785,33. Approved,3647415,Repair,Tail Light Assembly Replacement,105291.0,TOYOTA,TOYOTA LANDCRUISER PRADO,2020,Diesel,2.8,Auto,4WD,0.0,NSW,2,0,0,NaN,NaN,665.0
1,2172921,33. Approved,3647828,Prescribed,Prescribed,19957.0,ISUZU,ISUZU N-SERIES,2020,Diesel,NaN,Auto,2WD,NaN,NSW,2,0,0,60000.0,NaN,940.0
2,2173119,33. Approved,3648072,Log,Logbook Service,141320.0,HYUNDAI,HYUNDAI TUCSON,2021,Petrol,2.0,Auto,NaN,0.0,SA,2,0,0,135000.0,108.0,372.0
3,2173187,33. Approved,3648235,Repair,Roadworthy Inspection / Pink Slips,227817.0,ISUZU,ISUZU D-MAX,2015,Diesel,3.0,Auto,2WD,0.0,NSW,1,0,0,NaN,NaN,51.0
4,2173202,33. Approved,3648252,Repair,Balance Tyres,14528.0,SUBARU,SUBARU FORESTER,2024,Petrol,2.5,Auto,4WD,0.0,VIC,2,0,0,NaN,NaN,32.0


## Remove Duplicate Rows

In [16]:
# Duplicates can skew analysis and lead to incorrect conclusions, thus it is important each row is unique. Duplicates in this dataset are considered as rows that contain the same values across all columns other than the ID columns 
print(f"size before: {df.shape}" )
df = df.drop_duplicates(subset=[col for col in df.columns if col not in ['BookingID', 'BTicketID', 'BCreatedDateAEST', 'BStatusFromDateTimeAEST']])
print(f"size after: {df.shape}" )

size before: (900316, 21)
size after: (811258, 21)


## Remove False Negatives 

In [17]:
# False negatives are rows which have a status of '16. Requires Changes' when they should have a status of '33. Approved'. This occurs in the dataset as bookings contain multiple tickets. If a single ticket in a booking requires changes, the entire booking is marked as 'Requires Changes', thus marking tickets that do not require changes incorrectly. These false negatives can be detected and removed by checking if a duplicate entry exists where only the status changes.
print(f"size before: {df.shape}" )
df['StatusPriority'] = df['BStatusAfterSubmitted'].apply(lambda x: 0 if x == '33. Approved' else 1) # Assign priority: approved gets highest priority (lowest number)
dedup_cols = [col for col in df.columns if col not in ['BStatusAfterSubmitted', 'StatusPriority','BCreatedDateAEST', 'BStatusFromDateTimeAEST', 'Date' ]] # Define columns to check for duplicates
df = df.sort_values(by=dedup_cols + ['StatusPriority']) # Sort so approved status is first
df = df.drop_duplicates(subset=dedup_cols, keep='first') # Keep the first occurrence (which is the approved status)
df = df.drop(columns='StatusPriority') # drop the temporary column used for sorting

df['Label'] = df['BStatusAfterSubmitted'].map({'33. Approved': 1, '16. Requires Changes': 0, '29. Rejected': 0}) # create label column for model training
df = df.drop(columns=['BStatusAfterSubmitted'], axis=1) # drop the original status column as it is no longer needed
print(f"size after: {df.shape}" ) 

size before: (811258, 21)
size after: (704804, 21)


# Final Column Removal

In [18]:
columns_to_drop = ['BookingID', 'BTicketID', 'IsCustomService', 'IsCustomRepair', 'BShopRegionClass', 'BShopState']
df.drop(columns = columns_to_drop, inplace=True)
df.head()

,BTicketType,TaskName,Odometer,Make,Model,Year,FuelType,EngineSize,Transmission,DriveType,IsHybrid,Distance,Months,AdjustedPrice,Label
321331,Log,Logbook Service,NaN,SUBARU,SUBARU IMPREZA,2004,NaN,NaN,NaN,NaN,NaN,187500.0,90.0,285.0,0
322807,Log,Logbook Service,72199.0,SKODA,SKODA FABIA III,2017,Petrol,1.0,Auto,NaN,0.0,75000.0,60.0,710.0,1
322808,Log,Logbook Service,72199.0,SKODA,SKODA FABIA III,2017,Petrol,1.0,Auto,NaN,0.0,75000.0,60.0,786.0,0
322806,Log,Logbook Service,72199.0,SKODA,SKODA FABIA III,2017,Petrol,1.0,Auto,NaN,0.0,75000.0,60.0,854.0,0
317747,Log,Logbook Service,81168.0,MAZDA,MAZDA CX-9,2018,Petrol,2.5,Auto,2WD,0.0,80000.0,96.0,586.0,1


## Split data Frame by Ticket Type and Export

In [19]:
df_repair = df[df['BTicketType'] == 'Repair'].drop(columns=['BTicketType'])
df_log = df[df['BTicketType'] == 'Log'].drop(columns=['BTicketType'])
df_capped = df[df['BTicketType'] == 'Capped'].drop(columns=['BTicketType'])
df_prescribed = df[df['BTicketType'] == 'Prescribed'].drop(columns=['BTicketType'])

In [20]:
df_repair.to_csv('data/preprocessed_repair_data.csv', index=False)
df_log.to_csv('data/preprocessed_log_data.csv', index=False)
df_capped.to_csv('data/preprocessed_capped_data.csv', index=False)
df_prescribed.to_csv('data/preprocessed_prescribed_data.csv', index=False)